In [30]:
from PIL import Image, ImageEnhance, ImageFilter, ImageOps
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r"C:\\Program Files\\Tesseract-OCR\\tesseract.exe"

We are going to use pyteseract to extract min and max temps so we have the scale range.

In [31]:
def run_ocr_on_coords(image_path):
    # Hardcoded coordinates for min/max temp regions
    coords = {
        "min_top_left": {"x": 36, "y": 31, "width": 34, "height": 15},
        "max_top_left": {"x": 35, "y": 45, "width": 35, "height": 17},
        "max_right": {"x": 197, "y": 38, "width": 38, "height": 18},
        "min_right": {"x": 199, "y": 261, "width": 36, "height": 19}
    }

        # Load the image
    img = Image.open(image_path)

    # This dictionary will store for each region both the OCR text and its confidence
    results = {}

    for label, coord in coords.items():
        # Extract coordinates
        x, y, w, h = coord["x"], coord["y"], coord["width"], coord["height"]
        cropped = img.crop((x, y, x + w, y + h))
        
        # Preprocess: grayscale, contrast enhancement, and sharpening
        gray = ImageOps.grayscale(cropped)
        enhanced = ImageEnhance.Contrast(gray).enhance(2.0)
        sharp = enhanced.filter(ImageFilter.SHARPEN)
        
        # Use image_to_data to get detailed OCR results including confidence values
        data = pytesseract.image_to_data(
            sharp,
            output_type=pytesseract.Output.DICT,
            config='--psm 6 -c tessedit_char_whitelist=0123456789.'
        )
        
        best_text = ""
        best_conf = -1  # Initialize with -1 to handle non-detections
        # Iterate through detected words (or fragments)
        for i in range(len(data['text'])):
            text_candidate = data['text'][i].strip()
            try:
                # Confidence returned as string; convert it to a float
                conf = float(data['conf'][i])
            except ValueError:
                conf = -1  # or simply skip if conversion fails
            # Pick the candidate with the highest confidence
            if text_candidate and conf > best_conf:
                best_text = text_candidate
                best_conf = conf

        results[label] = {"value": best_text, "conf": best_conf}

    # For regions where the same value is expected (duplicates), select the one with higher confidence.
    def choose_better(region1, region2):
        r1 = results.get(region1)
        r2 = results.get(region2)
        if r1 and r2:
            return r1 if r1["conf"] >= r2["conf"] else r2
        return r1 or r2

    final_min = choose_better("min_top_left", "min_right")
    final_max = choose_better("max_top_left", "max_right")

    # Return final chosen values and associated confidence scores.
    return {
        "min": final_min["value"], "min_conf": final_min["conf"],
        "max": final_max["value"], "max_conf": final_max["conf"]
    }




# Example usage
if __name__ == "__main__":
    image_path = "thermal_imges_for_exp\CH01.jpeg"  # Replace with your path
    results = run_ocr_on_coords(image_path)
    for key, val in results.items():
        print(f"{key}: {val}")


min: 49.9
min_conf: 70.0
max: 71.8
max_conf: 85.0


In [32]:
from PIL import Image
import numpy as np
from scipy.spatial import KDTree

def smooth_white_pixels(image_path, output_path, k, tolerance,
                        smooth_temp_bar=True):
    """
    Smooths near-white and green pixels within specific bounding boxes and optionally smooths the temperature bar.
    """
    img = Image.open(image_path).convert("RGB")
    img_np = np.array(img)

    white_threshold = 255 - tolerance
    white_mask = np.all(img_np >= white_threshold, axis=-1)
    non_white_mask = ~white_mask

    white_coords = np.argwhere(white_mask)
    non_white_coords = np.argwhere(non_white_mask)

    allowed_boxes = [
        {'x': -1, 'y': 0, 'width': 71, 'height': 62},
        {'x': 166, 'y': 3, 'width': 70, 'height': 26},
        {'x': 197, 'y': 36, 'width': 38, 'height': 244},
        {'x': 177, 'y': 299, 'width': 54, 'height': 21},
        {'x': 0, 'y': 302, 'width': 46, 'height': 17},
    ]

    def is_in_any_box(y, x, boxes):
        for box in boxes:
            x0, y0 = box['x'], box['y']
            x1, y1 = x0 + box['width'], y0 + box['height']
            if y0 <= y < y1 and x0 <= x < x1:
                return True
        return False

    white_coords = np.array([coord for coord in white_coords if is_in_any_box(coord[0], coord[1], allowed_boxes)])

    if non_white_coords.size == 0:
        print("No non-white pixels found; nothing to average against.")
        return 0

    tree = KDTree(non_white_coords)
    smoothed_img = img_np.copy()
    changed_count = 0

    # Step 1: Smooth near-white pixels inside defined bounding boxes
    for y, x in white_coords:
        distances, indices = tree.query([y, x], k=k)
        nearest_pixels = non_white_coords[indices]
        pixel_values = np.array([img_np[ny, nx] for ny, nx in nearest_pixels])
        avg_color = pixel_values.mean(axis=0).astype(np.uint8)
        smoothed_img[y, x] = avg_color
        changed_count += 1

    # Step 2: Smooth green pixels in specified box using k-NN
    gx, gy, gw, gh = 99, 140, 38, 41
    green_coords = []
    green_box_mask = np.zeros_like(white_mask)
    
    for i in range(gy, gy+gh):
        for j in range(gx, gx+gw):
            r, g, b = img_np[i, j]
            if g > 100 and g > r and g > b:
                green_coords.append((i, j))
                green_box_mask[i, j] = True
    
    green_coords = np.array(green_coords)
    outside_mask = ~green_box_mask
    outside_coords = np.argwhere(outside_mask)

    if outside_coords.size > 0 and green_coords.size > 0:
        green_tree = KDTree(outside_coords)
        for y, x in green_coords:
            distances, indices = green_tree.query([y, x], k=min(k, len(outside_coords)))
            nearest_pixels = outside_coords[indices]
            pixel_values = np.array([img_np[ny, nx] for ny, nx in nearest_pixels])
            avg_color = pixel_values.mean(axis=0).astype(np.uint8)
            smoothed_img[y, x] = avg_color
            changed_count += 1

    # Step 3: Smooth temperature bar using k-NN
    if smooth_temp_bar:
        bar_coords={'x': 215, 'y': 56, 'width': 20, 'height': 209}
        img_height, img_width = smoothed_img.shape[:2]
        x, y, w, h = bar_coords['x'], bar_coords['y'], bar_coords['width'], bar_coords['height']
        x_end = min(x + w, img_width)
        y_end = min(y + h, img_height)

        all_coords = np.array([(i, j) for i in range(img_height) for j in range(img_width)])
        outside_mask = ~((all_coords[:, 0] >= y) & (all_coords[:, 0] < y_end) &
                         (all_coords[:, 1] >= x) & (all_coords[:, 1] < x_end))

        outside_coords = all_coords[outside_mask]
        inside_coords = all_coords[~outside_mask]

        if outside_coords.shape[0] == 0:
            print("No outside pixels found for smoothing the bar.")
        else:
            bar_tree = KDTree(outside_coords)
            for py, px in inside_coords:
                distances, indices = bar_tree.query([py, px], k=min(k, len(outside_coords)))
                nearest = outside_coords[indices]
                neighbor_colors = np.array([smoothed_img[ny, nx] for ny, nx in nearest])
                avg_color = neighbor_colors.mean(axis=0).astype(np.uint8)
                smoothed_img[py, px] = avg_color
            print("Temperature bar smoothed using external k-NN.")

    smoothed_image = Image.fromarray(smoothed_img)
    smoothed_image.save(output_path)

    print(f"Smoothed image saved to: {output_path}")
    print(f"Total number of pixels changed: {changed_count}")
    return changed_count


if __name__ == "__main__":
    # Specify the path to your input image and the output destination.
    image_path = "thermal_imges_for_exp\CH01.jpeg"          # Change to your image file path if needed.
    output_path = "thermal_imges_for_exp\CH01_smoothed.jpeg"  # The processed image will be saved here.
    
    smooth_white_pixels(image_path, output_path, k=150, tolerance=150)


Temperature bar smoothed using external k-NN.
Smoothed image saved to: thermal_imges_for_exp\CH01_smoothed.jpeg
Total number of pixels changed: 2584
